In [27]:
import snakes
import tpn
import snakes.plugins as pl
snakes.plugins.load([tpn, "gv"], "snakes.nets", "snk")
from snk import *
import time, random, sys


class Arc :
    def __init__(self, source, destination, orientation, value = Value(dot)) :
        self.source = source
        self.destination = destination
        self.orientation = orientation
        self.value = value

class PetriNetManager :
    def __init__(self, petriNetName) :
        self.petriNet = snk.PetriNet(petriNetName)
        
    def createTimedPN(self, min_times, max_times) : # create our model with the delays provided (should be ordered)
        # initialization of the PN elements
        #places
        producer = Place("producer",[dot])
        transmission_buffer = Place("transmission buffer",[])
        activation_place = Place("activation_place", [dot])
        reception_buffer = Place("reception buffer", [])
        buffer_vide = Place("buffer vide", [dot])
        buffer_plein = Place("buffer plein", [])
        consumer = Place("consumer", [dot])
            
        places = [producer, transmission_buffer, activation_place, reception_buffer, buffer_vide, buffer_plein, consumer]

        for place in places :
            self.petriNet.add_place(place)

        #transitions
        t1 = Transition("t1", min_time = min_times[0], max_time = max_times[0])
        t2 = Transition("t2", min_time = min_times[1], max_time = max_times[1])
        t3 = Transition("t3", min_time = min_times[2], max_time = max_times[2])
        t4 = Transition("t4", min_time = min_times[3], max_time = max_times[3])
        t5 = Transition("t5", min_time = min_times[4], max_time = max_times[4])
        transitions = [t1,t2,t3,t4,t5]

        for trans in transitions :
            self.petriNet.add_transition(trans)

        #arcs
        arc1 = Arc('producer', 't1', 'bidirectional')
        arc2 = Arc('transmission buffer', 't1', 'output')
        arc3 = Arc('activation_place', 't2', 'bidirectional')
        arc4 = Arc('transmission buffer', 't2', 'input')
        
        arc5 = Arc('reception buffer', 't2', 'output')
        arc6 = Arc('reception buffer', 't3', 'input')
        arc7 = Arc('reception buffer', 't4', 'input')
        
        arc8 = Arc('buffer vide', 't3', 'input')
        arc9 = Arc('buffer vide', 't5', 'output')
        
        arc10 = Arc('buffer plein', 't3', 'output')
        arc11 = Arc('buffer plein', 't4', 'bidirectional')
        arc12 = Arc('buffer plein', 't5', 'input')
        
        arc13 = Arc('consumer', 't5', 'bidirectional')

        arcsPN = [arc1, arc2, arc3, arc4, arc5, arc6, arc7, arc8, arc9, arc10, arc11, arc12, arc13]

        for arc in arcsPN : 
            if arc.orientation == 'input' :
                self.petriNet.add_input(arc.source, arc.destination, arc.value)
            elif arc.orientation == 'output' : 
                self.petriNet.add_output(arc.source, arc.destination, arc.value)
            elif arc.orientation == 'bidirectional':
                self.petriNet.add_input(arc.source, arc.destination, arc.value)
                self.petriNet.add_output(arc.source, arc.destination, arc.value)
    
    def runSimulation(self, simulation_time) :
        clock = 0.0
        list_dicts_trans = [{'name' : 't1', 'fired_times' : 0},
        {'name' : 't2', 'fired_times' : 0},
        {'name' : 't3', 'fired_times' : 0},
        {'name' : 't4', 'fired_times' : 0},
        {'name' : 't5', 'fired_times' : 0}
        ]
        #delay = 0.0
        self.petriNet.reset()
        
        #save max_times
        initial_max_times = []
        for t in self.petriNet.transition():
            initial_max_times.append(t.max_time)
        
        #initialize fire time set flag to indicate if a fire-time has been picked with the randomize function
        fireTimeSet = [{'name' : 't1', 'fireTimeSet' : False}, {'name' : 't2', 'fireTimeSet' : False},
                       {'name' : 't3', 'fireTimeSet' : False}, {'name' : 't4', 'fireTimeSet' : False},
                       {'name' : 't5', 'fireTimeSet' : False}]

        while clock < simulation_time :
            
            delay = self.petriNet.time()
            #time.sleep(delay/1000)
            clock += delay
            #print()
            #print('[{}], delay : {}'.format(clock, delay))
            
            #for t in self.petriNet.transition() :
                #print('{}[{},{}]={} , '.format(t,t.min_time, t.max_time, t.time), end='')
            #print()
            
            for t in self.petriNet.transition() :
                if t.enabled(t.modes()) :
                    if (t.min_time == t.max_time) or (t.time == t.max_time):

                        #fire transition
                        t.fire(t.modes())
                        #print('[{}] fire : {}'.format(clock, t))
                        #self.petriNet.draw("/home/mehdi/PetriNet model/Producer_consumer_tests/{}s-fire-{}.png".format(clock, t))
                        #reinitialize its max_time to the initial value
                        if(t.name == "t1") : 
                            #print('t1 reinitialize max_time')
                            t.max_time = initial_max_times[0]
                        elif (t.name == "t2"):
                            #print('t2 reinitialize max_time')
                            t.max_time = initial_max_times[1]
                        elif (t.name == "t3"):
                            #print('t3 reinitialize max_time')
                            t.max_time = initial_max_times[2]
                        elif (t.name == "t4"):
                            t.max_time = initial_max_times[3]
                        elif (t.name == "t5"):
                            t.max_time = initial_max_times[4]

                        #reinitialize the fireTimeSet boolean to False
                        next(item for item in fireTimeSet if item['name'] == str(t))['fireTimeSet'] = False
                        #count the firing times of each transition
                        next(item for item in list_dicts_trans if item['name'] == str(t))['fired_times'] += 1 

                    else :
                        #check if the fire time has been set before or not
                        if next(item for item in fireTimeSet if item['name'] == str(t))['fireTimeSet'] == False :
                            n = random.randint(t.time, t.max_time)
                            t.max_time = n
                            next(item for item in fireTimeSet if item['name'] == str(t))['fireTimeSet'] = True
                         #   print('rand = {}.max_time = {}'.format(t, n))

            #print('tokens[transmission_buffer] = {} | tokens[buffer_plein] = {}'.format(len(self.petriNet.place('transmission buffer').tokens),len(self.petriNet.place('buffer plein').tokens)))
            
        return(list_dicts_trans)

    def getTransitionsDelays(self) : 
        min_delays = []
        max_delays = []
        for t in self.petriNet.transition() : 
            min_delays.append(t.min_time)
            max_delays.append(t.max_time)
        return min_delays, max_delays

    def updateTransitionsDelays(self, min_times, max_times) :
        i = 0
        if((len(self.petriNet.transition()) == len(min_times)) & (len(min_times) == len(max_times))) :
            for t in self.petriNet.transition() :
                t.min_time = min_times[i]
                t.max_time = max_times[i]
                i +=1
        else : # define exception
            print('length of the parameters do not correspond to the number of transitions')
    
    def updateTransitionDelay(self, transition, min_time, max_time) :
        transition.min_time = min_time
        transition.max_time = max_time
if __name__ == '__main__':
    min_times = [50, 8, 0, 0, 0]
    max_times = [50, 79, 0, 0, 50]
    
    pnManager = PetriNetManager("PetriNetNDT")
    #pnManager2 = PetriNetManager(PetriNet("PetriNDT2"))
    pnManager.createTimedPN(min_times, max_times)
    #pnManager.petriNet.draw("/home/mehdi/PetriNet model/producer_consumer.png")
    pnManager.petriNet.reset()
    stats = pnManager.runSimulation(10000)
    print(stats)
    #fired_times_t1 = next(item for item in stats if item['name'] == 't1')['fired_times']
    #fired_times_t3 = next(item for item in stats if item['name'] == 't3')['fired_times']
    #print("fired_times_t3/fired_times_t1 = {}".format(fired_times_t3/fired_times_t1))

[{'name': 't1', 'fired_times': 200}, {'name': 't2', 'fired_times': 198}, {'name': 't3', 'fired_times': 158}, {'name': 't4', 'fired_times': 40}, {'name': 't5', 'fired_times': 158}]


In [29]:
delays_compacted = [[[17, 38], [17, 38], [18, 38], [18, 38], [18, 38]],
[[17, 38], [18, 40], [20, 42], [20, 42], [20, 42]],
[[18, 39], [19, 42], [20, 45], [20, 45], [20, 45]],
[[19, 40], [19, 44], [37, 143], [38, 165],[38, 170]],
[[23, 67], [25, 68], [30, 115], [170, 415], [175, 420]]
]
delays_minmax = [[[8, 68], [8, 69], [8, 68], [8, 68], [8, 68]],
[[7, 71], [7, 74], [8, 79], [8, 80], [10, 80]],
[[8, 72], [6, 79], [6, 84], [6, 85], [6, 85]],
[[8, 73], [8, 81], [8, 315], [8, 365], [8, 370]],
[[8, 130], [8, 131], [8, 245], [8, 780], [8, 790]]
]
retries = [0, 1, 2, 5, 7]
data_rates = [0.5, 0.2, 0.1, 0.07, 0.05]


In [32]:

pnManager = PetriNetManager("PetriNetNDT")
pnManager.createTimedPN([0, 0, 0, 0, 0], [0, 0, 0, 0, 0])
for dr_index in range(0, len(data_rates)) : 
    print("DSI = {}".format(data_rates[dr_index]))
    for retry_index in range(0, len(retries)) :
        print("Retries = {}".format(retries[retry_index]))
        min_delay = delays_compacted[dr_index][retry_index][0]
        #min_delay = delays_minmax[dr_index][retry_index][0]
        max_delay = delays_compacted[dr_index][retry_index][1]
        #max_delay = delays_minmax[dr_index][retry_index][1]
        min_times = [data_rates[dr_index]*1000/2, min_delay, 0, 0, 0]
        max_times = [data_rates[dr_index]*1000/2, max_delay, 0, 0, data_rates[dr_index]*1000/2]
        pnManager.petriNet.reset()
        pnManager.updateTransitionsDelays(min_times, max_times)
        setMin_times, setMax_times = pnManager.getTransitionsDelays()
        print("min_times = {}, max_times = {}".format(setMin_times, setMax_times))
        stats = pnManager.runSimulation(1500000)
        print(stats)
        fired_times_t1 = next(item for item in stats if item['name'] == 't1')['fired_times']
        fired_times_t4 = next(item for item in stats if item['name'] == 't4')['fired_times']
        fired_times_t5 = next(item for item in stats if item['name'] == 't5')['fired_times']
        #Loss = fired_times_t4/fired_times_t1
        PDR = 1-(fired_times_t4/fired_times_t1)
        print("PDR = {}".format(PDR))
        #print("Packet loss = {}".format(Loss))
        print()


DSI = 0.5
Retries = 0
min_times = [250.0, 17, 0, 0, 0], max_times = [250.0, 38, 0, 0, 250.0]
[{'name': 't1', 'fired_times': 6000}, {'name': 't2', 'fired_times': 5999}, {'name': 't3', 'fired_times': 5892}, {'name': 't4', 'fired_times': 107}, {'name': 't5', 'fired_times': 5892}]
PDR = 0.9821666666666666

Retries = 1
min_times = [250.0, 17, 0, 0, 0], max_times = [250.0, 38, 0, 0, 250.0]
[{'name': 't1', 'fired_times': 6000}, {'name': 't2', 'fired_times': 6000}, {'name': 't3', 'fired_times': 5901}, {'name': 't4', 'fired_times': 99}, {'name': 't5', 'fired_times': 5901}]
PDR = 0.9835

Retries = 2
min_times = [250.0, 18, 0, 0, 0], max_times = [250.0, 38, 0, 0, 250.0]
[{'name': 't1', 'fired_times': 6000}, {'name': 't2', 'fired_times': 6000}, {'name': 't3', 'fired_times': 5894}, {'name': 't4', 'fired_times': 106}, {'name': 't5', 'fired_times': 5894}]
PDR = 0.9823333333333333

Retries = 5
min_times = [250.0, 18, 0, 0, 0], max_times = [250.0, 38, 0, 0, 250.0]
[{'name': 't1', 'fired_times': 6000}, 